In [ ]:
from flask import Flask, request
import json
import logging
from datetime import datetime

app = Flask(__name__)

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

@app.route('/meter', methods=['GET'])
def receive_meter_data():
    try:
        # Log incoming request details
        logger.info("=== INCOMING GET REQUEST ===")
        logger.info(f"Method: {request.method}")
        logger.info(f"Full URL: {request.url}")
        logger.info(f"Query String: {request.query_string}")
        logger.info(f"Remote IP: {request.remote_addr}")
        logger.info(f"User-Agent: {request.headers.get('User-Agent', 'Unknown')}")
        
        # Get query parameters (matching the C code parameter names)
        voltage = request.args.get('v')          # Changed from 'volt' to 'v'
        current = request.args.get('c')          # Changed from 'curr' to 'c' 
        power_factor = request.args.get('pf')    # Same as before
        load_kw = request.args.get('l')          # Changed from 'load' to 'l'
        kwh = request.args.get('k')              # Changed from 'kwh' to 'k'
        frequency = request.args.get('f')        # Changed from 'freq' to 'f'
        datetime_str = request.args.get('d')     # Changed from 'dt' to 'd'
        retry_count = request.args.get('r', '0') # Changed from 'retry' to 'r'
        source = request.args.get('s', '')       # Added source parameter
        
        # Log raw query parameters
        logger.info(f"Raw query params: {dict(request.args)}")
        
        # Check if this is a boot message (based on source parameter)
        if source and 'boot' in source.lower():
            print("\n" + "🚀"*20)
            print("🔄 SYSTEM BOOT NOTIFICATION")
            print("🚀"*20)
            print(f"📱 Device: {source}")
            print(f"🕒 Boot Time: {datetime_str}")
            print(f"🔄 Previous Failures: {retry_count}")
            print(f"🌐 From IP: {request.remote_addr}")
            print(f"⏰ Server Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            print("🚀"*20 + "\n")
            return "BOOT_ACK", 200
        
        # Check for missing required data
        missing_fields = []
        if not voltage: missing_fields.append('v (voltage)')
        if not current: missing_fields.append('c (current)')
        if not kwh: missing_fields.append('k (kWh)')
        
        if missing_fields:
            error_msg = f"Missing required fields: {', '.join(missing_fields)}"
            logger.error(error_msg)
            return f"ERROR: {error_msg}", 400
        
        # Display meter data with enhanced formatting
        print("\n" + "⚡"*60)
        print("🔌 SMART ENERGY METER DATA RECEIVED")
        print("⚡"*60)
        print(f"📊 Voltage      : {voltage}V")
        print(f"⚡ Current      : {current}A") 
        print(f"📈 Power Factor : {power_factor}")
        print(f"🔋 Load         : {load_kw}kW")
        print(f"📋 Total kWh    : {kwh}")
        print(f"🌊 Frequency    : {frequency}Hz")
        print(f"🕒 DateTime     : {datetime_str}")
        print(f"🔄 Retry Count  : {retry_count}")
        print(f"🏷️  Source       : {source}")
        print(f"🌐 From IP      : {request.remote_addr}")
        print(f"⏰ Server Time  : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print("⚡"*60)
        
        # Validate numeric data with appropriate ranges for energy meters
        try:
            if voltage: 
                volt_val = float(voltage)
                if volt_val < 0 or volt_val > 500:
                    raise ValueError(f"Voltage {volt_val}V out of range (0-500V)")
                    
            if current: 
                curr_val = float(current)
                if curr_val < 0 or curr_val > 1000:
                    raise ValueError(f"Current {curr_val}A out of range (0-1000A)")
                    
            if power_factor:
                pf_val = float(power_factor)
                if pf_val < 0 or pf_val > 1:
                    raise ValueError(f"Power Factor {pf_val} out of range (0-1)")
                    
            if load_kw:
                load_val = float(load_kw)
                if load_val < 0:
                    raise ValueError(f"Load {load_val}kW cannot be negative")
                    
            if kwh: 
                kwh_val = float(kwh)
                if kwh_val < 0:
                    raise ValueError(f"kWh {kwh_val} cannot be negative")
                    
            if frequency:
                freq_val = float(frequency)
                if freq_val < 45 or freq_val > 65:
                    raise ValueError(f"Frequency {freq_val}Hz out of range (45-65Hz)")
                    
            print("✅ Data validation PASSED")
            logger.info("Data validation successful")
            
        except (ValueError, TypeError) as e:
            error_msg = f"Invalid data format: {str(e)}"
            print(f"❌ Data validation FAILED: {error_msg}")
            logger.error(error_msg)
            return f"ERROR: {error_msg}", 400
        
        # Success response
        print("✅ Data processed successfully\n")
        logger.info("GET request processed successfully")
        return "OK", 200
        
    except Exception as e:
        error_msg = f"Server error: {str(e)}"
        print(f"🔥 EXCEPTION: {error_msg}")
        logger.error(f"Unexpected error: {error_msg}")
        return f"ERROR: {error_msg}", 500

@app.route('/health', methods=['GET'])
def health_check():
    return {
        "status": "healthy",
        "timestamp": datetime.now().isoformat(),
        "server": "Smart Energy Meter Data Receiver",
        "endpoint": "/meter (GET)",
        "expected_params": {
            "v": "voltage (float)",
            "c": "current (float)", 
            "pf": "power_factor (float)",
            "l": "load_kw (float)",
            "k": "kwh (float)",
            "f": "frequency (float)",
            "d": "datetime (string)",
            "r": "retry_count (integer)",
            "s": "source (string)"
        }
    }, 200

@app.route('/test', methods=['GET'])
def test_endpoint():
    """Test endpoint to verify server connectivity"""
    return {
        "message": "Smart Energy Meter Server is running",
        "timestamp": datetime.now().isoformat(),
        "test_url": "/meter?v=230.5&c=8.750&pf=0.92&l=2.01560&k=1250.75&f=50.2&d=26-07-2025%2013:05:30&r=0&s=atmega328pb"
    }, 200

@app.errorhandler(404)
def not_found(error):
    logger.warning(f"404 error: {request.url}")
    return {
        "error": "Endpoint not found",
        "message": "Use GET /meter with query parameters",
        "example": "/meter?v=230&c=8.5&k=1250&pf=0.92&l=2.0&f=50.2&d=26-07-2025%2013:05:30&r=0&s=atmega328pb"
    }, 404

@app.errorhandler(405)
def method_not_allowed(error):
    logger.warning(f"405 error: {request.method} to {request.url}")
    return {
        "error": f"Method {request.method} not allowed",
        "message": "Use GET method for /meter endpoint with query parameters"
    }, 405

if __name__ == '__main__':
    print("🚀 Starting Smart Energy Meter Data Server...")
    print("📡 Listening for meter data on /meter endpoint (GET)")
    print("💡 Expected URL format:")
    print("   /meter?v=230.5&c=8.750&pf=0.92&l=2.01560&k=1250.75&f=50.2&d=26-07-2025%2013:05:30&r=0&s=atmega328pb")
    print("🏥 Health check available on /health endpoint")
    print("🧪 Test endpoint available on /test endpoint")
    print("⚙️  Parameter mapping:")
    print("   v  = voltage (V)")
    print("   c  = current (A)")
    print("   pf = power factor")
    print("   l  = load (kW)")
    print("   k  = kWh total")
    print("   f  = frequency (Hz)")
    print("   d  = datetime")
    print("   r  = retry count") 
    print("   s  = source device")
    app.run(host='0.0.0.0', port=8080)

🚀 Starting Smart Energy Meter Data Server...
📡 Listening for meter data on /meter endpoint (GET)
💡 Expected URL format:
   /meter?v=230.5&c=8.750&pf=0.92&l=2.01560&k=1250.75&f=50.2&d=26-07-2025%2013:05:30&r=0&s=atmega328pb
🏥 Health check available on /health endpoint
🧪 Test endpoint available on /test endpoint
⚙️  Parameter mapping:
   v  = voltage (V)
   c  = current (A)
   pf = power factor
   l  = load (kW)
   k  = kWh total
   f  = frequency (Hz)
   d  = datetime
   r  = retry count
   s  = source device
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://172.20.10.5:8080
INFO:werkzeug:Press CTRL+C to quit
INFO:__main__:=== INCOMING GET REQUEST ===
INFO:__main__:Method: GET
INFO:__main__:Full URL: http://fffe6d0846f3.ngrok-free.app/meter?v=230.5&c=8.750&pf=0.92&l=2.01560&k=1250.75&f=50.2&d=26-07-2025%2013:05:30&r=0&s=atmega328pb
INFO:__main__:Query String: b'v=230.5&c=8.750&pf=0.92&l=2.01560&k=1250.75&f=50.2&d=26-07-2025%2013:05:30&r=0&s=atmega328pb'
INFO:__main__:Remote IP: 127.0.0.1
INFO:__main__:User-Agent: curl/8.13.0
INFO:__main__:Raw query params: {'v': '230.5', 'c': '8.750', 'pf': '0.92', 'l': '2.01560', 'k': '1250.75', 'f': '50.2', 'd': '26-07-2025 13:05:30', 'r': '0', 's': 'atmega328pb'}
INFO:__main__:Data validation successful
INFO:__main__:GET request processed successfully
INFO:werkzeug:127.0.0.1 - - 


⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡
🔌 SMART ENERGY METER DATA RECEIVED
⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡
📊 Voltage      : 230.5V
⚡ Current      : 8.750A
📈 Power Factor : 0.92
🔋 Load         : 2.01560kW
📋 Total kWh    : 1250.75
🌊 Frequency    : 50.2Hz
🕒 DateTime     : 26-07-2025 13:05:30
🔄 Retry Count  : 0
🏷️  Source       : atmega328pb
🌐 From IP      : 127.0.0.1
⏰ Server Time  : 2025-07-26 14:16:59
⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡
✅ Data validation PASSED
✅ Data processed successfully



INFO:__main__:=== INCOMING GET REQUEST ===
INFO:__main__:Method: GET
INFO:__main__:Full URL: http://fffe6d0846f3.ngrok-free.app/meter?v=254.10&c=0.000&pf=1.00&l=0.00000&k=12.66&f=50.1&d=26-07-25%2015:55:01&s=atmega328pb
INFO:__main__:Query String: b'v=254.10&c=0.000&pf=1.00&l=0.00000&k=12.66&f=50.1&d=26-07-25%2015:55:01&s=atmega328pb'
INFO:__main__:Remote IP: 127.0.0.1
INFO:__main__:User-Agent: Unknown
INFO:__main__:Raw query params: {'v': '254.10', 'c': '0.000', 'pf': '1.00', 'l': '0.00000', 'k': '12.66', 'f': '50.1', 'd': '26-07-25 15:55:01', 's': 'atmega328pb'}
INFO:__main__:Data validation successful
INFO:__main__:GET request processed successfully
INFO:werkzeug:127.0.0.1 - - [26/Jul/2025 15:58:12] "GET /meter?v=254.10&c=0.000&pf=1.00&l=0.00000&k=12.66&f=50.1&d=26-07-25%2015:55:01&s=atmega328pb HTTP/1.1" 200 -



⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡
🔌 SMART ENERGY METER DATA RECEIVED
⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡
📊 Voltage      : 254.10V
⚡ Current      : 0.000A
📈 Power Factor : 1.00
🔋 Load         : 0.00000kW
📋 Total kWh    : 12.66
🌊 Frequency    : 50.1Hz
🕒 DateTime     : 26-07-25 15:55:01
🔄 Retry Count  : 0
🏷️  Source       : atmega328pb
🌐 From IP      : 127.0.0.1
⏰ Server Time  : 2025-07-26 15:58:12
⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡
✅ Data validation PASSED
✅ Data processed successfully



...


⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡
🔌 SMART ENERGY METER DATA RECEIVED
⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡
📊 Voltage      : 256.05V
⚡ Current      : 0.000A
📈 Power Factor : 1.00
🔋 Load         : 0.00000kW
📋 Total kWh    : 12.66
🌊 Frequency    : 50.1Hz
🕒 DateTime     : 26-07-25 15:57:59
🔄 Retry Count  : 0
🏷️  Source       : atmega328pb
🌐 From IP      : 127.0.0.1
⏰ Server Time  : 2025-07-26 16:01:09
⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡
✅ Data validation PASSED
✅ Data processed successfully



INFO:__main__:=== INCOMING GET REQUEST ===
INFO:__main__:Method: GET
INFO:__main__:Full URL: http://fffe6d0846f3.ngrok-free.app/meter?v=255.56&c=0.000&pf=1.00&l=0.00000&k=12.66&f=50.1&d=26-07-25%2015:58:08&s=atmega328pb
INFO:__main__:Query String: b'v=255.56&c=0.000&pf=1.00&l=0.00000&k=12.66&f=50.1&d=26-07-25%2015:58:08&s=atmega328pb'
INFO:__main__:Remote IP: 127.0.0.1
INFO:__main__:User-Agent: Unknown
INFO:__main__:Raw query params: {'v': '255.56', 'c': '0.000', 'pf': '1.00', 'l': '0.00000', 'k': '12.66', 'f': '50.1', 'd': '26-07-25 15:58:08', 's': 'atmega328pb'}
INFO:__main__:Data validation successful
INFO:__main__:GET request processed successfully
INFO:werkzeug:127.0.0.1 - - [26/Jul/2025 16:01:18] "GET /meter?v=255.56&c=0.000&pf=1.00&l=0.00000&k=12.66&f=50.1&d=26-07-25%2015:58:08&s=atmega328pb HTTP/1.1" 200 -



⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡
🔌 SMART ENERGY METER DATA RECEIVED
⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡
📊 Voltage      : 255.56V
⚡ Current      : 0.000A
📈 Power Factor : 1.00
🔋 Load         : 0.00000kW
📋 Total kWh    : 12.66
🌊 Frequency    : 50.1Hz
🕒 DateTime     : 26-07-25 15:58:08
🔄 Retry Count  : 0
🏷️  Source       : atmega328pb
🌐 From IP      : 127.0.0.1
⏰ Server Time  : 2025-07-26 16:01:18
⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡
✅ Data validation PASSED
✅ Data processed successfully



INFO:__main__:=== INCOMING GET REQUEST ===
INFO:__main__:Method: GET
INFO:__main__:Full URL: http://fffe6d0846f3.ngrok-free.app/meter?v=254.72&c=0.000&pf=1.00&l=0.00000&k=12.66&f=50.2&d=26-07-25%2015:58:17&s=atmega328pb
INFO:__main__:Query String: b'v=254.72&c=0.000&pf=1.00&l=0.00000&k=12.66&f=50.2&d=26-07-25%2015:58:17&s=atmega328pb'
INFO:__main__:Remote IP: 127.0.0.1
INFO:__main__:User-Agent: Unknown
INFO:__main__:Raw query params: {'v': '254.72', 'c': '0.000', 'pf': '1.00', 'l': '0.00000', 'k': '12.66', 'f': '50.2', 'd': '26-07-25 15:58:17', 's': 'atmega328pb'}
INFO:__main__:Data validation successful
INFO:__main__:GET request processed successfully
INFO:werkzeug:127.0.0.1 - - [26/Jul/2025 16:01:27] "GET /meter?v=254.72&c=0.000&pf=1.00&l=0.00000&k=12.66&f=50.2&d=26-07-25%2015:58:17&s=atmega328pb HTTP/1.1" 200 -



⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡
🔌 SMART ENERGY METER DATA RECEIVED
⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡
📊 Voltage      : 254.72V
⚡ Current      : 0.000A
📈 Power Factor : 1.00
🔋 Load         : 0.00000kW
📋 Total kWh    : 12.66
🌊 Frequency    : 50.2Hz
🕒 DateTime     : 26-07-25 15:58:17
🔄 Retry Count  : 0
🏷️  Source       : atmega328pb
🌐 From IP      : 127.0.0.1
⏰ Server Time  : 2025-07-26 16:01:27
⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡
✅ Data validation PASSED
✅ Data processed successfully

